In [1]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [2]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/amrragababdelaziz/dermnet-and-skin-disease-9-classes')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 3.51G/3.51G [00:18<00:00, 204MB/s]


In [3]:
train ='/content/dermnet-and-skin-disease-9-classes/Merged Dataset/Train'
validation_dir = '/content/dermnet-and-skin-disease-9-classes/Merged Dataset/test'

In [4]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [5]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

9 Classes
26642 Train images
2389 Test images


In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
img_width,img_height =224,224
input_shape=(img_width,img_height,3)
batch_size =32
train_generator =train_datagen.flow_from_directory(train,
                                target_size=(img_width,img_height),batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(validation_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 26642 images belonging to 9 classes.
Found 2389 images belonging to 9 classes.


In [7]:
od.download('https://www.kaggle.com/datasets/keras/inceptionresnetv2')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 392M/392M [00:03<00:00, 120MB/s]


In [10]:
from keras import Model
from keras import optimizers
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dense

model_dir = "../content/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
model = Sequential()
model.add( tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights= model_dir, input_shape=(224, 244, 3)))

model.add(GlobalAveragePooling2D())
model.add(Dense(9, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 6, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense (Dense)               (None, 9)                 13833     
                                                                 
Total params: 54,350,569
Trainable params: 54,290,025
Non-trainable params: 60,544
_________________________________________________________________


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(train_generator,validation_data=test_generator,epochs=50,shuffle=True)

Epoch 1/50
833/833 [==============================] - 765s 795ms/step - loss: 1.2219 - accuracy: 0.5631 - val_loss: 1.6784 - val_accuracy: 0.4152
Epoch 2/50
833/833 [==============================] - 647s 777ms/step - loss: 1.0067 - accuracy: 0.6445 - val_loss: 1.4884 - val_accuracy: 0.4864
Epoch 3/50
833/833 [==============================] - 636s 763ms/step - loss: 0.8695 - accuracy: 0.6941 - val_loss: 1.2155 - val_accuracy: 0.5877
Epoch 4/50
151/833 [====>.........................] - ETA: 8:16 - loss: 0.7552 - accuracy: 0.7305

In [ ]:
model.save('inceptionresnetv2_model.h5')